# ML stroke prediction - Logistic Regression
dataset link: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset?resource=download

## Classification of stroke or no-stroke

This notebook will go further on the logistic regression by calculating an F1 score.

In [1]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np
np.set_printoptions(suppress=True)

import csv
import sys

# SKlearn for F1 score calculation
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
from scipy.io import loadmat

# we use pandas to import a comma-seperated values dataset
import pandas as pd

# tells matplotlib to embed plots within the notebook
%matplotlib inline

### First importing the dataset and small conversions

For the first implementation we're just using a limited set of features for the model.

In [2]:
#  training data stored in arrays X, y
#df = pd.read_csv(os.path.join('data', 'healthcare-dataset-stroke-data.csv'))
#X = pd.DataFrame(df, columns=['age', 'hypertension', 'heart_disease', 'avg_glucose_level']).to_numpy()
#X = np.around(X, 5)
#print("X: ")
#print(X)

#y = pd.DataFrame(df, columns=['stroke']).to_numpy()
#y = np.around(y, 5)
#y.reshape(5110)
#print("y: ")
#print(y)
 
data = pd.read_csv(os.path.join('data', 'healthcare-dataset-stroke-data.csv'))

data.head(10)

data.drop("id", axis = 1, inplace = True)
data.drop("gender", axis = 1, inplace = True)
data.drop("ever_married", axis = 1, inplace = True)
data.drop("work_type", axis = 1, inplace = True)
data.drop("Residence_type", axis = 1, inplace = True)
data.drop("bmi", axis = 1, inplace = True)
data.drop("smoking_status", axis = 1, inplace = True)

print(data)

X = data.drop(['stroke'], axis=1).values
y = data['stroke'].values

print("Xshape: ")
print(X.shape)
print("Yshape: ")
print(y.shape)

print("X: ")
print(X)

print("y: ")
print(y)

       age  hypertension  heart_disease  avg_glucose_level  stroke
0     67.0             0              1             228.69       1
1     61.0             0              0             202.21       1
2     80.0             0              1             105.92       1
3     49.0             0              0             171.23       1
4     79.0             1              0             174.12       1
...    ...           ...            ...                ...     ...
5105  80.0             1              0              83.75       0
5106  81.0             0              0             125.20       0
5107  35.0             0              0              82.99       0
5108  51.0             0              0             166.29       0
5109  44.0             0              0              85.28       0

[5110 rows x 5 columns]
Xshape: 
(5110, 4)
Yshape: 
(5110,)
X: 
[[ 67.     0.     1.   228.69]
 [ 61.     0.     0.   202.21]
 [ 80.     0.     1.   105.92]
 ...
 [ 35.     0.     0.    82.99]
 [

The sigmoid function copied from excersise 2. Could be copied later on to a 'library'.

In [3]:
def sigmoid(z):
    
    # convert input to a numpy array
    z = np.array(z)
    
    # You need to return the following variables correctly 
    g = np.zeros(z.shape)

    # ====================== YOUR CODE HERE ======================

    g = 1/(1 + np.exp(-z))

    # =============================================================
    return g

### Preparing parameters
For the cost function we need the following parameters:
- Theta
- X
- y

Also we need to ad a X0 (offset) row in the feature matrix.

In [4]:
# Setup the data matrix appropriately, and add ones for the intercept term
m, n = X.shape

print(X.shape)

# Add intercept term to X
X = np.concatenate([np.ones((m, 1)), X], axis=1)

print(X.shape)
#print(X)

(5110, 4)
(5110, 5)


In [5]:
_lambda = 0.2

In [6]:
def lrCostFunction(theta, X, y, lambda_):
    #Initialize some useful values
    m = y.size
    
    # convert labels to ints if their type is bool
    if y.dtype == bool:
        y = y.astype(int)
    
    # You need to return the following variables correctly
    J = 0
    grad = np.zeros(theta.shape)
    
    # ====================== YOUR CODE HERE ======================

    H = sigmoid(np.matmul(X, theta))
    J = (sum(-y*np.log(H)-(1-y)*np.log(1-H)))/(m)+((lambda_/(2*m))*sum(theta[1:]**2))
    
    diff = np.subtract(H, y)
    
    grad = ((np.matmul(np.transpose(X), diff))/m) + ((lambda_ * theta)/m)
    grad[0]= ((np.matmul(np.transpose(X), diff))/m)[0]
        
    # =============================================================
    return J, grad

In [7]:
# Initialize fitting parameters
initial_theta = np.zeros(n+1)
cost, grad = lrCostFunction(initial_theta, X, y, _lambda)
#print(cost)
#print(grad)

#print(cost.shape)
#print(grad.shape)

print('Cost at test theta: {:.3f}'.format(cost))

print('Gradient at test theta:')
print('\t[{:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}]'.format(*grad))

Cost at test theta: 0.693
Gradient at test theta:
	[0.451, 18.313, 0.036, 0.018, 46.615]


In [8]:
# set options for optimize.minimize
options= {'maxiter': 500}

# see documention for scipy's optimize.minimize  for description about
# the different parameters
# The function returns an object `OptimizeResult`
# We use truncated Newton algorithm for optimization which is 
# equivalent to MATLAB's fminunc
# See https://stackoverflow.com/questions/18801002/fminunc-alternate-in-numpy

#res = optimize.minimize(lrCostFunction,
#                        initial_theta,
#                        (X, y, _lambda),
#                        jac=True,
#                        method='TNC',
#                        options=options)

result = optimize.fmin_tnc(func=lrCostFunction, x0=initial_theta, args=(X, y, _lambda))

# the fun property of `OptimizeResult` object returns
# the value of costFunction at optimized theta
#cost = res.fun

# the optimized theta is in the x property
#theta = res.x

theta = result[0]
(cost, grad) = lrCostFunction(theta, X, y, _lambda)

# Print theta to screen
print('Cost at theta found by optimize.minimize: {:.3f}'.format(cost))

print('theta:')
print('\t[{:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}]'.format(*theta))

Cost at theta found by optimize.minimize: 0.156
theta:
	[-7.489, 0.069, 0.380, 0.328, 0.004]


In [9]:
def predict(theta, X):
    m = X.shape[0] # Number of training examples

    # You need to return the following variables correctly
    p = np.zeros(m)

    # ====================== YOUR CODE HERE ======================

    p = sigmoid(np.matmul(X, theta))
    
    #np.set_printoptions(threshold=sys.maxsize)
    #print(p)
    
    p [p >= 0.5] = 1
    p [p < 0.5]  = 0
    
    
    # ============================================================
    return p

In [10]:
#  Predict probability for a person with age 80, hypertension, heart disease, and avg glucose of 228  
prob = sigmoid(np.dot([1, 80, 1, 1, 228], theta))
print('For this person,'
      'we predict a stroke probability of {:.3f}'.format(prob))

# Compute accuracy on our training set
p = predict(theta, X)
print('Train Accuracy: {:.2f} %'.format(np.mean(p == y) * 100))

For this person,we predict a stroke probability of 0.419
Train Accuracy: 95.13 %


In [11]:
counter_tp = 0
counter_p =0
for i in range(0, y.size -1):
    if p[i] ==1:
        counter_p +=1
        if y[i]==1:
            counter_p += 1
            counter_tp += 1
        
print(counter_tp)
print(counter_p)

0
0


In [12]:
print(f1_score(y,p,average='binary'))
p[0]=1
print(f1_score(y,p,average='binary'))
p[1]=1
print(f1_score(y,p,average='binary'))
p[2]=1
print(f1_score(y,p,average='binary'))
p[3]=1
print(f1_score(y,p,average='binary'))
p[4]=1
print(f1_score(y,p,average='binary'))
p[5:249]=1
print(f1_score(y,p,average='binary'))
p[1423]=1
print(f1_score(y,p,average='binary'))

0.0
0.007999999999999998
0.015936254980079678
0.02380952380952381
0.03162055335968379
0.03937007874015748
1.0
0.9979959919839679
